In [2]:
import dynamicrouting_summary as dr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import oursin as urchin
import numpy as np

In [3]:
dfs = dr.get_dfs()

In [4]:
dfs.keys()

KeysView(LazyDict(keys=['session', 'subject', 'epochs', 'trials', 'performance', 'vis_rf_mapping', 'aud_rf_mapping', 'optotagging', 'invalid_times', 'devices', 'electrode_groups', 'electrodes']))

In [5]:
electrodes_original = dfs['electrodes']
electrodes_original

,location,group_name,structure,x,y,z,channel,rel_x,rel_y,reference,imp,session_idx,date,subject_id,session_id,is_ephys,is_templeton,is_training,is_dynamic_routing,is_opto
0,fiber tracts,probeB,fiber tracts,8200.0,3575.0,3050.0,0,27,0,tip,150000.0,0,2022-07-26,620263,620263_2022-07-26_0,True,True,False,False,False
1,fiber tracts,probeB,fiber tracts,8200.0,3550.0,3050.0,1,59,0,tip,150000.0,0,2022-07-26,620263,620263_2022-07-26_0,True,True,False,False,False
2,fiber tracts,probeB,fiber tracts,8200.0,3550.0,3050.0,2,11,20,tip,150000.0,0,2022-07-26,620263,620263_2022-07-26_0,True,True,False,False,False
3,fiber tracts,probeB,fiber tracts,8200.0,3525.0,3050.0,3,43,20,tip,150000.0,0,2022-07-26,620263,620263_2022-07-26_0,True,True,False,False,False
4,fiber tracts,probeB,fiber tracts,8200.0,3525.0,3050.0,4,27,40,tip,150000.0,0,2022-07-26,620263,620263_2022-07-26_0,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163579,unannotated,probeF,None,NaN,NaN,NaN,379,43,3780,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
163580,unannotated,probeF,None,NaN,NaN,NaN,380,27,3800,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
163581,unannotated,probeF,None,NaN,NaN,NaN,381,59,3800,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
163582,unannotated,probeF,None,NaN,NaN,NaN,382,11,3820,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False


In [6]:
import pyarrow.parquet as pq
import npc_lims

units = pq.ParquetDataset(npc_lims.get_cache_path('units'))
units.schema

num_spikes: double
firing_rate: double
presence_ratio: double
snr: double
isi_violations_ratio: double
isi_violations_count: double
rp_contamination: double
rp_violations: double
sliding_rp_violation: double
amplitude_cutoff: double
drift_ptp: double
drift_std: double
drift_mad: double
isolation_distance: double
l_ratio: double
d_prime: double
peak_to_valley: double
peak_trough_ratio: double
half_width: double
repolarization_slope: double
recovery_slope: double
electrode_group_name: string
peak_channel: int64
cluster_id: int64
default_qc: bool
amplitude: double
channels: list<element: int64>
  child 0, element: int64
unit_id: string
group_name: string
ccf_ap: double
ccf_dv: double
ccf_ml: double
structure: string
location: string
peak_electrode: int64
spike_times: list<element: double>
  child 0, element: double
obs_intervals: list<element: list<element: double>>
  child 0, element: list<element: double>
      child 0, element: double
electrodes: list<element: int64>
  child 0, element

In [7]:
units_structures = units.read(columns=['structure', 'location', 'peak_channel', 'subject_id', 'session_idx', 'date', 
                                       'ccf_ap', 'ccf_dv', 'ccf_ml']).to_pandas()
units_structures_with_bools = dr.add_bool_columns(units_structures)
units_structures_with_bools.dropna(inplace=True)
units_structures_with_bools.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'session_id', 'is_ephys',
       'is_templeton', 'is_training', 'is_dynamic_routing', 'is_opto'],
      dtype='object')

In [8]:
import ipywidgets

electrodes = electrodes_original.dropna()
#electrodes = electrodes[~(electrodes['session_id'].str.contains('662892'))]
electrodes = electrodes[~(electrodes['structure'].str.islower())]
electrodes = electrodes[electrodes['is_dynamic_routing'] == True]

In [10]:
units_structures_with_bools_dr = units_structures_with_bools[units_structures_with_bools['is_dynamic_routing'] == True]
units_structures_with_bools_dr.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'session_id', 'is_ephys',
       'is_templeton', 'is_training', 'is_dynamic_routing', 'is_opto'],
      dtype='object')

In [36]:
from ipywidgets import interact

In [38]:
def get_subject_colors(electrodes_structures_area: pd.DataFrame) -> list[tuple[int, int, int]]: 
    subject_colors = dr.utils.generate_subject_random_colors(electrodes_structures_area)
    subject_color_list = []

    for index, row in electrodes_structures_area.iterrows():
        subject_color_list.append(subject_colors[str(row['subject_id'])])
    
    return subject_color_list

In [52]:
unique_areas = electrodes['structure'].unique()
unique_areas.sort()
area_dropdown = ipywidgets.Dropdown(options=unique_areas, value=unique_areas[0], description='CCF Structure Area', disabled=False)
area_dropdown

@interact(x=unique_areas)
def update_3d_viewer(x):
    urchin.ccf25.clear()
    urchin.clear_meshes()
    units_structures_with_bools_dr_area = units_structures_with_bools_dr[units_structures_with_bools_dr['structure'] == x]
    electrodes_structures_area = electrodes[electrodes['structure'] == x]

    print(f"{x}: {len(electrodes_structures_area['subject_id'].unique())} mice")
    print(f"{x}: {len(electrodes_structures_area['session_id'].unique())} sessions")
    print(f"{x}: {len(units_structures_with_bools_dr_area)} units")

    pmeshes = urchin.meshes.create(len(electrodes_structures_area)) #creates 2 primitives, stored in list pmeshes
    coords = electrodes_structures_area[['x', 'z', 'y']].to_numpy().tolist()


    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_structures_area))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas([x])

    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")


interactive(children=(Dropdown(description='x', options=('ACAd', 'ACAv', 'ACB', 'AD', 'AId', 'AIv', 'AON', 'AP…

In [40]:
urchin.setup()

(urchin) Client is already connected. Use ID: 05c6fc5b


In [44]:
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [34]:
@interact(x=['coronal', 'axial', 'sagittal'])
def camera_rotation(x):
    urchin.camera.main.set_rotation(x)
    urchin.camera.main.set_mode('perspective')
    urchin.camera.main.set_background_color('#ffffff')

interactive(children=(Dropdown(description='x', options=('coronal', 'axial', 'sagittal'), value='coronal'), Ou…

In [53]:
@interact(x=(5, 60))
def zoom(x=45):
    urchin.camera.main.set_zoom(60-x)

interactive(children=(IntSlider(value=45, description='x', max=60, min=5), Output()), _dom_classes=('widget-in…

(URN) disconnected from server
(URN) connected to server
Login sent with ID: 05c6fc5b, copy this ID into the renderer to connect.
(URN) disconnected from server
(URN) connected to server
Login sent with ID: 05c6fc5b, copy this ID into the renderer to connect.


In [47]:
counts = {'area': [], 'num_units': [], 'num_sessions': [], 'num_subjects': []}
units_structure_areas_no_white_matter = units_structures_with_bools_dr[~(units_structures_with_bools_dr['structure'].str.islower())]
for area in units_structure_areas_no_white_matter['structure'].unique():
    units_structure_areas = units_structure_areas_no_white_matter[units_structure_areas_no_white_matter['structure'] == area]
    counts['area'].append(area)
    counts['num_sessions'].append(len(units_structure_areas['session_id'].unique()))
    counts['num_subjects'].append(len(units_structure_areas['subject_id'].unique()))
    counts['num_units'].append(len(units_structure_areas))

df_counts = pd.DataFrame(counts)

In [48]:
df_counts.style.applymap(lambda x: 'background-color : red' if x>=3 else '', subset=['num_subjects']).applymap(lambda x: 'background-color : red' if x>150 else '', subset=['num_units'])

C:\Users\arjun.sridhar\AppData\Local\Temp\ipykernel_10340\2511661253.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_counts.style.applymap(lambda x: 'background-color : red' if x>=3 else '', subset=['num_subjects']).applymap(lambda x: 'background-color : red' if x>150 else '', subset=['num_units'])


,area,num_units,num_sessions,num_subjects
0,AUDp,204,3,3
1,AUDpo,247,4,3
2,BST,176,7,5
3,CA1,3379,29,12
4,CA3,1284,23,11
5,CP,4762,31,13
6,DG,1989,33,12
7,IGL,34,2,2
8,LD,681,10,7
9,LGd,433,7,6
